<a href="https://colab.research.google.com/github/McAjumma/Coursera_Capstone/blob/main/Toronto_Neighborhoods2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping the Data

In this section the data is scraped from the Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. This is used to create a table that stores the postal code, borough, and neighboors.

In [46]:
import numpy as np
import pandas as pd

import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

This is grabbing everything from the webpage.

In [47]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
hmtl_data = requests.get(url).text

This is changing everything that was grabbed to be read as html code.

In [48]:
soup = BeautifulSoup(hmtl_data, "html5lib")

This section starts by creating an empty list to store the data and load into the DataFrame t_n. Next the code searches for the a table in the html data code, storing it in table. The for loop will go through each line. The loop will skip if the line has 'Not assigned' as a value. If the line doesn't have that it will change the line to text, remove unneeded characters and store it in the made list. After looping through the whole table the list is stored into a DataFrame. The last line cleans up the names of some of the boroughs stored in the DataFrame.

In [49]:
table_contents=[]
#t_n = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhoods']
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
t_n=pd.DataFrame(table_contents)
t_n['Borough']=t_n['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

This is showing the newly created DataFrame.

In [50]:
t_n

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


And this is the shape of the Dataframe.

In [51]:
t_n.shape

(103, 3)

# Latitude and Longitude

This section grabs the Latitude and Longitude from the Geospatial_Coordinates.csv and stores it in a DataFrame. From there it merges the new DataFrame and the one from the previous section.

The CSV file, Geospatial_Coordinates.csv, is store into a DataFrame. The columns are renamed to be used in the next section. The newly form DataFrame is displayed.

In [ ]:
la_lng = pd.read_csv("Geospatial_Coordinates.csv")
names = ['PostalCode', 'Latitude', 'Longitude']
la_lng.columns = names
la_lng

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


This merges the two Dataframe on the column "PostalCode".

In [ ]:
t_neghborhood = pd.merge(t_n, la_lng, on="PostalCode")
t_neghborhood

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
